In [1]:
!pip install neo4j

In [18]:
!pip install python-dotenv

In [33]:
import json
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# Neo4j 연결 설정 (환경 변수에서 읽어오기)
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

# Neo4j 데이터베이스에 연결
driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

# JSON 파일 경로들
json_files = [
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/01_num_cal.json",
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/02_change_of_relationship.json",
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/03_shape_meas.json",
    "D:/programming/python/chunjae/finalproject/kst_pipeline/Preprocessing/04_data_and_possibility.json",
]


# 대분류 노드를 생성하는 함수
def create_category_node(name):
    query = """
    MERGE (c:Category {name: $name})
    RETURN c
    """
    session.run(query, name=name)


# 세부 항목 노드를 생성하는 함수
def create_item_node(name, parent_name=None):
    query = """
    MERGE (i:Item {name: $name})
    """
    if parent_name:
        query += """
        WITH i
        MATCH (p {name: $parent_name})
        MERGE (p)-[:HAS_CHILD]->(i)
        """
    session.run(query, name=name, parent_name=parent_name)


# JSON 파일을 읽고 노드를 삽입하는 함수
def insert_json_data(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

        # 대분류 노드 생성
        for key, value in data.items():
            category_name = value["name"]
            create_category_node(category_name)

            # 세부 항목을 처리하여 노드 삽입
            def process_items(data, parent_name=None):
                for key, value in data.items():
                    if isinstance(value, dict):
                        item_name = value["name"]
                        # 항목 노드 생성
                        create_item_node(item_name, parent_name=parent_name)

                        # 자식 항목이 있을 경우 재귀적으로 처리
                        if "children" in value:
                            process_items(value["children"], item_name)
                    else:
                        # Handle case where value is not a dict but a direct string
                        item_name = value
                        create_item_node(item_name, parent_name=parent_name)

            process_items(value["children"], parent_name=category_name)


# 각 JSON 파일을 순차적으로 처리
for file_path in json_files:
    # JSON 파일 데이터를 Neo4j에 삽입
    insert_json_data(file_path)

# Neo4j 세션 종료
session.close()